In [4]:
# !pip install tensorflow librosa matplotlib joblib glob

In [20]:
import numpy as np
import os
import matplotlib.pyplot as plt
import librosa
import joblib
from glob import glob
import IPython.display as ipd

In [65]:
example_audio = glob('data/notScreaming/*.wav')[0]

In [73]:
# Resample the audio to a different sample rate
resampled_audio, _ = librosa.load(example_audio, sr=4000)
ipd.Audio(resampled_audio, rate=4000)
# ipd.Audio(example_audio)

Ler os arquivos de áudio para coletar os coeficientes e os os labels. Os labels serão divididos da seguinte maneira:
- Anger: 0
- Disgust: 1
- Fear: 2
- Happy: 3
- Neutral: 4
- Sad: 5

In [43]:
sr = 4000
x, sr = librosa.load(example_audio, sr=sr)
x = librosa.util.fix_length(x, size=sr*10)
print(x, sr, len(x))

librosa.feature.mfcc(y=x, sr=sr, n_mfcc=64).shape

[2.2189579e-09 9.1747104e-10 1.5234889e-09 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] 4000 40000


(64, 79)

In [3]:
base_dir = 'data/'

In [4]:
max_len = max([librosa.get_duration(path=audio_file) for audio_file in glob('./data/' + 'notScreaming' + '/*.wav')] + [librosa.get_duration(path=audio_file) for audio_file in glob('./data/' + 'screaming' + '/*.wav')])
min_len = min([librosa.get_duration(path=audio_file) for audio_file in glob('./data/' + 'notScreaming' + '/*.wav')] + [librosa.get_duration(path=audio_file) for audio_file in glob('./data/' + 'screaming' + '/*.wav')])
print('min_index: ', ([librosa.get_duration(path=audio_file) for audio_file in glob('./data/' + 'notScreaming' + '/*.wav')] + [librosa.get_duration(path=audio_file) for audio_file in glob('./data/' + 'screaming' + '/*.wav')]).index(min_len))
min_len, max_len

min_index:  737


(0.5108616780045352, 10.0)

In [46]:
label_map = {'notScreaming': 0, 'screaming': 1}
lst = []
number_of_audios_per_class = -1 # set -1 for all audios
sr = 4000
n_mfcc = 64

for emotion in label_map.keys():
    folder_path = os.path.join(base_dir, emotion)
    audios_number = 0
    # Iterate over the audio files in the folder
    for audio_file in glob('./data/' + emotion + '/*.wav'):
        # print(os.path.join(folder_path, audio_file))
        x, sample_rate = librosa.load(audio_file, sr=sr)
        x = librosa.util.fix_length(x, size=sr*10)
        mfccs = np.array(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=n_mfcc))
        label = label_map[emotion]
        arr = mfccs, label
        lst.append(arr)
        audios_number += 1
        if(audios_number == number_of_audios_per_class): break
        print(f'\r{audios_number} {emotion} audio files loaded', end='')
    print(audios_number, emotion, 'audio files loaded')

2631 notScreaming audio files loaded2631 notScreaming audio files loaded
862 screaming audio files loaded862 screaming audio files loaded


In [49]:
x, y = zip(*lst)

In [55]:
x = np.asarray(x)
y = np.asarray(y)


x.shape, y.shape

((3493, 64, 79), (3493,))

In [5]:
import joblib

In [61]:


X_name = 'x.joblib'
y_name = 'y.joblib'
save_dir = 'data'

savedX = joblib.dump(x, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [45]:
x = joblib.load('data/x.joblib')
y = joblib.load('data/y.joblib')

In [92]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

## Neural Network

In [93]:
# x_traincnn = np.expand_dims(X_train, axis=2)
# x_testcnn = np.expand_dims(X_test, axis=2)

x_traincnn = X_train
x_testcnn = X_test

In [94]:
x_traincnn.shape, x_testcnn.shape

((1746, 64, 79), (1747, 64, 79))

In [15]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# List available GPUs
print("Available GPUs: ", tf.config.experimental.list_physical_devices('GPU'))

Num GPUs Available:  0
Available GPUs:  []


## Modelo 1



Acuracia de 0.87

In [50]:
callback = keras.callbacks.EarlyStopping(monitor='loss',patience= 10)
# callbacks = [
#     keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=10),
# ]
model = Sequential()
model.add(Conv1D(64, 5, padding='same', input_shape=(64, 79)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])

cnn_history = model.fit(x_traincnn, y_train,
                batch_size=16, epochs=1000,
                validation_data=(x_testcnn, y_test), callbacks=[callback])

Epoch 1/1000


c:\Users\alech\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


147/147 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7265 - loss: 9.4177 - val_accuracy: 0.7971 - val_loss: 4.8116
Epoch 2/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8167 - loss: 3.4200 - val_accuracy: 0.7511 - val_loss: 4.0387
Epoch 3/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8325 - loss: 2.5780 - val_accuracy: 0.7944 - val_loss: 3.8106
Epoch 4/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8795 - loss: 1.8185 - val_accuracy: 0.7918 - val_loss: 4.2114
Epoch 5/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8964 - loss: 1.2157 - val_accuracy: 0.7988 - val_loss: 4.0646
Epoch 6/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9102 - loss: 1.3709 - val_accuracy: 0.7936 - val_loss: 8.2274
Epoch 7/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9069 - loss: 1.5496 - val_accuracy: 0.8031 - val_loss: 7.6202
Epoch 8/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9270 - loss: 1.3161 - val_accuracy:

In [51]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [52]:
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.78      0.83       863
           1       0.53      0.73      0.61       290

    accuracy                           0.77      1153
   macro avg       0.71      0.75      0.72      1153
weighted avg       0.80      0.77      0.78      1153



## Modelo https://www.kaggle.com/code/shivamburnwal/speech-emotion-recognition?scriptVersionId=34958802&cellId=52

Acurácia de 0.79

In [53]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [82]:
model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(64, 79)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(units=2, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

c:\Users\alech\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)              │ (None, 64, 256)        │       101,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 32, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 32, 256)        │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 16, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 16, 128)        │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 8, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 8, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 8, 64)          │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,746 (2.46 MB)

 Trainable params: 643,170 (2.45 MB)

 Non-trainable params: 576 (2.25 KB)

In [83]:
from tensorflow.keras.utils import to_categorical

In [84]:
y_train_encoded = to_categorical(y_train, num_classes=2)
y_test_encoded = to_categorical(y_test, num_classes=2)

In [85]:
y_train_encoded.shape

(2340, 2)

In [86]:
callback = keras.callbacks.EarlyStopping(monitor='loss',patience= 3)
history=model.fit(x_traincnn, y_train_encoded, batch_size=64, epochs=100, validation_data=(x_testcnn, y_test_encoded), callbacks=[callback])

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 90ms/step - accuracy: 0.5661 - loss: 0.7703 - val_accuracy: 0.7329 - val_loss: 0.7399
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.7325 - loss: 0.5519 - val_accuracy: 0.8612 - val_loss: 0.3777
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.8216 - loss: 0.4206 - val_accuracy: 0.8031 - val_loss: 0.4852
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.8732 - loss: 0.3532 - val_accuracy: 0.8569 - val_loss: 0.3552
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 121ms/step - accuracy: 0.8845 - loss: 0.3016 - val_accuracy: 0.7051 - val_loss: 0.5679
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 137ms/step - accuracy: 0.8801 - loss: 0.3100 - val_accuracy: 0.8049 - val_loss: 0.5139
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 121ms/step - accuracy: 0.8797 - loss: 0.2935 - val_accuracy: 0.8621 - val_loss: 0.4418
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.9160 - loss: 0.2436 - val_accuracy

In [87]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


In [88]:
predictions_encoded = to_categorical(predictions, num_classes=2)

In [89]:
from sklearn.metrics import classification_report
report = classification_report(y_test_encoded, predictions_encoded)
print(report)

              precision    recall  f1-score   support

           0       0.87      0.93      0.90       863
           1       0.74      0.60      0.66       290

   micro avg       0.85      0.85      0.85      1153
   macro avg       0.81      0.77      0.78      1153
weighted avg       0.84      0.85      0.84      1153
 samples avg       0.85      0.85      0.85      1153



              precision    recall  f1-score   support

           0       0.87      0.94      0.90       863
           1       0.75      0.58      0.65       290

   micro avg       0.85      0.85      0.85      1153
   macro avg       0.81      0.76      0.78      1153
weighted avg       0.84      0.85      0.84      1153
 samples avg       0.85      0.85      0.85      1153

In [90]:
MODEL_DIR_PATH = 'models'

In [91]:
if not os.path.isdir(MODEL_DIR_PATH):
    os.makedirs(MODEL_DIR_PATH)
model_path = os.path.join(MODEL_DIR_PATH, 'scream_detection.keras')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at models\scream_detection.keras 
